In [1]:
import transformers
import json
import sys
import os
import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'
config_file_location = '../data/config.json'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

def update_config(checkpoint, file_location = '../data/config.json'):
    config_file = open(file_location, 'r')
    config = json.load(config_file)
    print(config)
    config["_name_or_path"] = checkpoint
    config_file = open(file_location, 'w')
    json.dump(config, config_file, indent= 2)
    config_file.close()

Reusing dataset squad (/soe/joalara/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [2]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val, lr):
    !CUDA_VISIBLE_DEVICES=1 python run_qa_alt.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/full_squad_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 40\
      --per_device_eval_batch_size 40\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate {lr} \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir \
      --train_adapter \
      --adapter_config houlsby \
      --load_adapter @ukp/roberta-base_qa_squad1_houlsby

In [3]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-22b6c9f63817822a
Reusing dataset squad (/soe/joalara/.cache/huggingface/datasets/squad/default-22b6c9f63817822a/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-c26cd1994268e9f2
Reusing dataset squad (/soe/joalara/.cache/huggingface/datasets/squad/default-c26cd1994268e9f2/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [4]:
def train_adapter(lr):
    k_fold = 5
    for i in range(k_fold):
        covid_fold = covid_qa.shard(k_fold, i)

        covid_test = covid_fold.shard(2, 0)
        covid_val = covid_fold.shard(2, 1)
        covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

        #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

        checkpoint = 'roberta-base'
        cur_dir = '../models/adapter_baseline/split_' + str(i)

        #log_file = open(cur_dir + "/log_file.txt","w+")
        #sys.stdout = log_file

        squad_qa.shuffle()
        bio_qa.shuffle()
        full_dataset = datasets.concatenate_datasets([bio_qa, covid_train])
        make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)
        run_gradual_ft(cur_dir, checkpoint, covid_val, lr)


In [ ]:
train_adapter(1e-5)
train_adapter(2e-5)
train_adapter(3e-5)
train_adapter(5e-5)

Loading cached shuffled indices for dataset at /soe/joalara/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0818905460d158f7.arrow


07/26/2021 16:47:35 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/26/2021 16:47:35 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|loading.py:76] 2021-07-26 16:47:43,319 >> Loading module configuration from ~/.cache/torch/adapters/ee464f64e055b7b434f3f133d27c4f12ab29a53e427275020c25232ebcaff680-78424428eab1232aefb47c1358bc02042157efc9cdcff01a0da0d066dea810cd-extracted/adapter_config.json
[INFO|configuration.py:260] 2021-07-26 16:47:43,320 >> Adding adapter 'full_squad_covidQA'.
[INFO|loading.py:135] 2021-07-26 16:47:43,453 >> Loading module weights from ~/.cache/torch/adapters/ee464f64e055b7b434f3f133d27c4f12ab29a53e427275020c25232ebcaff680-78424428eab1232aefb47c1358bc02042157efc9cdcff01a0da0d066dea810cd-extracted/pytorch_adapter.bin
[INFO|loading.py:76] 2021-07-26 16:47:43,464 >> Loading module configuration from ~/.cache/torch/adapters/ee464f64e055b7b434f3f133d27c4f12ab29a53e427275020c25232ebcaff680-78424428eab1232aefb47c1358bc02042157efc9cdcff01a0da0d066dea810cd-extracted/head_config.json
[WARNING|loading.py:655] 2021-07-26 16:47:43,465 >> Model class 'RobertaModelWithHeads' of found prediction head does 

In [ ]:
def train_adapter_base(lr):
    k_fold = 5
    for i in range(k_fold):
        covid_fold = covid_qa.shard(k_fold, i)

        covid_test = covid_fold.shard(2, 0)
        covid_val = covid_fold.shard(2, 1)
        covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

        #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

        checkpoint = 'roberta-base'
        cur_dir = '../models/adapter_COVID_baseline/split_' + str(i)

        #log_file = open(cur_dir + "/log_file.txt","w+")
        #sys.stdout = log_file

        squad_qa.shuffle()
        bio_qa.shuffle()
        make_and_save_full_dataset(covid_train, covid_val, covid_test, covid_and_squad_dataset_path)
        run_gradual_ft(cur_dir, checkpoint, covid_val, lr)


In [ ]:
train_adapter_base(1e-5)
train_adapter_base(2e-5)
train_adapter_base(3e-5)
train_adapter_base(5e-5)